## Estimating lifespan normative models

This notebook provides a complete walkthrough for an analysis of normative modelling using your own dataset. The tutorial is based on the teaching material at: https://github.com/CharFraza/CPC_ML_tutorial/. Here you can also find more normative modelling tutorials.
Training and testing data is provided for this tutorial. However, the idea is that you could subsitute our provided training and testing datasets for you own dataset (as long as it matches the same format!)

First, if necessary, we install PCNtoolkit (note: this tutorial requires at least version 0.20)

In [ ]:
# Make sure to click the restart runtime button at the
# bottom of this code blocks' output (after you run the cell)
! pip install pcntoolkit==0.28

Then we import the required libraries

In [2]:
! git clone https://github.com/CharFraza/CPC_ML_tutorial.git

Cloning into 'CPC_ML_tutorial'...


In [1]:
# we need to be in the CPC_ML_tutorial folder when we import the libraries in the code block below,
# because there is a function called nm_utils that is in this folder that we need to import
import os
os.chdir('CPC_ML_tutorial')

In [2]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns

from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d

Now, we configure the locations in which the data are stored.

**Notes:**
- The data are assumed to be in CSV format and will be loaded as pandas dataframes
- Generally the raw data will be in a different location to the analysis
- The data can have arbitrary columns but some are required by the script, i.e. 'age', 'sex' and 'site', plus the phenotypes you wish to estimate (see below)

In [ ]:
# where the raw data are stored
data_dir = 'data'

# where the analysis takes place
root_dir = 'D:/python_code/NM_educational_OHBM24-main/NM_educational_OHBM24-main/slot1_Fraza/gmv_test'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

Now we load the data.

We will load one pandas dataframe for the training set and one dataframe for the test set. We also configure a list of site ids.

In [4]:
"""work1 code"""
train = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\GMV_health_ins2.csv')
test = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\GMV_disease_ins2.csv')
Site = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\Site_chu.csv')
Site2 = Site[['eid', 'site']]
label = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\GMV_labels.csv')
train.columns = train.columns[:1].tolist() + label['Description'].to_list() + train.columns[-2:].tolist()
test.columns = test.columns[:1].tolist() + label['Description'].to_list() + test.columns[-2:].tolist()
train = pd.merge(train, Site2, on='eid')
test = pd.merge(test, Site2, on='eid')

In [6]:
train.to_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\train_data.csv')
test.to_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1\test_data.csv')

In [22]:
import pandas as pd
df_tr = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1_gmv\train_data.csv', index_col=0)
df_te = pd.read_csv(r'D:\python_code\NM_educational_OHBM24-main\NM_educational_OHBM24-main\slot1_Fraza\data_UKBB1_gmv\test_data.csv', index_col=0)

# extract a list of unique site ids from the training set
site_ids = sorted(set(df_tr['site'].to_list()))

### Configure which models to fit

Next, we load the image derived phenotypes (IDPs) which we will process in this analysis. This is effectively just a list of columns in your dataframe. Here we estimate normative models for the left hemisphere, right hemisphere and cortical structures.

In [23]:
# we choose here to process all idps. Uncomment lines 2-7 (and comment line 11) to run models for the whole brain, but we suggest just starting with several ROIs
#os.chdir(root_dir)
#!wget -nc https://raw.githubusercontent.com/CharFraza/CPC_ML_tutorial/master/data/task1_phenotypes.txt
#with open(os.path.join(root_dir,'task1_phenotypes.txt')) as f:
#  idp_ids = f.read().splitlines()
#for idx, ele in enumerate(idp_ids):
#        idp_ids[idx] = ele.replace('\t', '')

# we could also just specify a list of IDPs. Use this line to run just 2 models (not the whole brain)...this is a good place to start. If you have time,
# you can uncomment the above line and run the whole brain models. Be sure to comment out this line if you uncomment the above line.
idp_ids = df_tr.columns[4:]

### Configure model parameters

Now, we configure some parameters for the regression model we use to fit the normative model. Here we will use a 'warped' Bayesian linear regression model. To model non-Gaussianity, we select a sin arcsinh warp and to model non-linearity, we stick with the default value for the basis expansion (a cubic b-spline basis set with 5 knot points). Since we are sticking with the default value, we do not need to specify any parameters for this, but we do need to specify the limits. We choose to pad the input by a few years either side of the input range. We will also set a couple of options that control the estimation of the model

For further details about the likelihood warping approach, see [Fraza et al 2021](https://www.biorxiv.org/content/10.1101/2021.04.05.438429v1).

In [24]:
# check the min & max age of the dataset, use this info to update the xmin & xmax variables in the code block below.
df_tr['age'].describe()

count    24014.000000
mean        63.297021
std          7.446508
min         45.166667
25%         57.333333
50%         63.583333
75%         69.083333
max         82.250000
Name: age, dtype: float64

In [25]:
# which data columns do we wish to use as covariates?
# You could add additional covariates from your own dataset here that you wish to use as predictors.
# However, for this tutorial today we will keep it simple and just use age & sex.
# Maybe discuss with your partner ideas you have for other covariates you would like to include.
cols_cov = ['age','sex']

# which warping function to use? We can set this to None in order to fit a vanilla Gaussian noise model
warp =  'WarpSinArcsinh'
#warp = None

# limits for cubic B-spline basis
# check the min & max ages of the dataframes, add 5 to the max
# and subtract 5 from the min and adjust these variables accordingly
xmin = 45 # set this variable
xmax = 83 # set this variable

# Do we want to force the model to be refit every time?
# When training normative model from scratch like we are doing in this notebook (not re-using a pre-trained model),
# this variable should be = True
force_refit = True

# Absolute Z treshold above which a sample is considered to be an outlier (without fitting any model)
outlier_thresh = 7

### Fit the models

Now we fit the models. This involves looping over the IDPs we have selected. We will use a module from PCNtoolkit to set up the design matrices, containing the covariates, fixed effects for site and nonlinear basis expansion.

In [ ]:
for idp_num, idp in enumerate(idp_ids):
    print('Running IDP', idp_num, idp, ':')

    idp_dir = os.path.join(out_dir, idp)
    os.makedirs(idp_dir, exist_ok=True)
    os.chdir(idp_dir)

    # extract the response variables for training and test set
    y_tr = df_tr[idp].to_numpy()
    y_te = df_te[idp].to_numpy()

    # remove gross outliers and implausible values
    yz_tr = (y_tr - np.mean(y_tr)) / np.std(y_tr)
    yz_te = (y_te - np.mean(y_te)) / np.std(y_te)
    nz_tr = np.bitwise_and(np.abs(yz_tr) < outlier_thresh, y_tr > 0)
    nz_te = np.bitwise_and(np.abs(yz_te) < outlier_thresh, y_te > 0)
    y_tr = y_tr[nz_tr]
    y_te = y_te[nz_te]

    eid = df_te['eid'][nz_te]
    eid_file_te = os.path.join(idp_dir, 'eid.txt')
    np.savetxt(eid_file_te, eid, fmt='%d')

    # write out the response variables for training and test
    resp_file_tr = os.path.join(idp_dir, 'resp_tr.txt')
    resp_file_te = os.path.join(idp_dir, 'resp_te.txt')
    np.savetxt(resp_file_tr, y_tr)
    np.savetxt(resp_file_te, y_te)

    # configure the design matrix
    X_tr = create_design_matrix(df_tr[cols_cov].loc[nz_tr],
                                site_ids = df_tr['site'].loc[nz_tr],
                                basis = 'bspline',
                                p = 3,
                                nknots = 4,
                                xmin = xmin,
                                xmax = xmax)
    X_te = create_design_matrix(df_te[cols_cov].loc[nz_te],
                                site_ids = df_te['site'].loc[nz_te],
                                basis = 'bspline',
                                p = 3,
                                nknots = 4,
                                xmin = xmin,
                                xmax = xmax)

    # configure and save the covariates
    cov_file_tr = os.path.join(idp_dir, 'cov_bspline_tr.txt')
    cov_file_te = os.path.join(idp_dir, 'cov_bspline_te.txt')
    np.savetxt(cov_file_tr, X_tr)
    np.savetxt(cov_file_te, X_te)

    if not force_refit and os.path.exists(os.path.join(idp_dir, 'Models', 'NM_0_0_estimate.pkl')):
        print('Making predictions using a pre-existing model...')
        suffix = 'predict'

        # Make prdictsion with test data
        predict(cov_file_te,
                alg='blr',
                respfile=resp_file_te,
                model_path=os.path.join(idp_dir,'Models'),
                outputsuffix=suffix)
    else:
        print('Estimating the normative model...')
        estimate(cov_file_tr, resp_file_tr, testresp=resp_file_te,
                 testcov=cov_file_te, alg='blr', optimizer = 'powell', cvfolds=10,
                 savemodel=True, warp=warp, warp_reparam=True) 
        suffix = 'estimate'

## Questions
1. Can you explain the covariance matrix and it's elements?
2. How do you change the order of the spline function?
3. How do you change the number of knots?


In [ ]:
print(pd.DataFrame(X_te))

### Compute error metrics

In this section we compute the following error metrics for all IDPs (all evaluated on the test set): assess the goodness of fit between the predicted probabilities of a model and the actual observed outcomes.
- Negative log likelihood (NLL): NLL assesses the goodness of fit between the predicted probabilities of a model and the actual observed outcomes. In this case, it measures the discrepancy between the predicted probabilities of the model for the IDPs (Independent Data Points) and the actual outcomes on the test set.
- Explained variance (EV): EV assesses how much of the total variation in the dependent variable (IDP) is explained by the independent variables. In the context of this analysis, it quantifies the extent to which the independent variables account for the variability observed in the IDPs on the test set.
- Mean standardized log loss (MSLL): MSLL takes into account both the mean error and the estimated prediction variance. It is used to evaluate the performance of the model, and in this case, a lower MSLL indicates a better-fitting model for the IDPs on the test set.
- Bayesian information criteria (BIC): BIC is a model selection criterion that balances the goodness of fit to the data with the model's complexity. It penalizes models with higher flexibility and aims to find the best trade-off. Lower BIC scores indicate models that better explain the IDPs on the test set while considering the model complexity.
- Skew and Kurtosis of the Z-distribution: Skewness and kurtosis are statistical measures used to assess the shape and characteristics of a distribution. They provide information about how well the warping function performed in terms of capturing the departure from a normal distribution for the IDPs.

For more information on the different model selection criteria see [Fraza et al 2021](https://www.biorxiv.org/content/10.1101/2021.04.05.438429v1)

In [ ]:
# initialise dataframe we will use to store quantitative metrics
blr_metrics = pd.DataFrame(columns = ['eid', 'NLL', 'EV', 'MSLL', 'BIC','Skew','Kurtosis'])

for idp_num, idp in enumerate(idp_ids):
    print('Running IDP', idp_num, idp, ':')

    idp_dir = os.path.join(out_dir, idp)

    # load the predictions and true data. We use a custom function that ensures 2d arrays
    # equivalent to: y = np.loadtxt(filename); y = y[:, np.newaxis]
    yhat_te = load_2d(os.path.join(idp_dir, 'yhat_' + suffix + '.txt'))
    s2_te = load_2d(os.path.join(idp_dir, 'ys2_' + suffix + '.txt'))
    y_te = load_2d(os.path.join(idp_dir, 'resp_te.txt'))

    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle)

    # compute error metrics
    if warp is None:
        metrics = evaluate(y_te, yhat_te)

        # compute MSLL manually as a sanity check
        y_tr_mean = np.array( [[np.mean(y_tr)]] )
        y_tr_var = np.array( [[np.var(y_tr)]] )
        MSLL = compute_MSLL(y_te, yhat_te, s2_te, y_tr_mean, y_tr_var)
    else:
        warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1]
        W = nm.blr.warp

        # warp predictions
        med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
        med_te = med_te[:, np.newaxis]

        # evaluation metrics
        metrics = evaluate(y_te, med_te)

        # compute MSLL manually
        y_te_w = W.f(y_te, warp_param)
        y_tr_w = W.f(y_tr, warp_param)
        y_tr_mean = np.array( [[np.mean(y_tr_w)]] )
        y_tr_var = np.array( [[np.var(y_tr_w)]] )
        MSLL = compute_MSLL(y_te_w, yhat_te, s2_te, y_tr_mean, y_tr_var)

    Z = np.loadtxt(os.path.join(idp_dir, 'Z_' + suffix + '.txt'))
    [skew, sdskew, kurtosis, sdkurtosis, semean, sesd] = calibration_descriptives(Z)

    BIC = len(nm.blr.hyp) * np.log(y_tr.shape[0]) + 2 * nm.neg_log_lik

    blr_metrics.loc[len(blr_metrics)] = [idp, nm.neg_log_lik, metrics['EXPV'][0],
                                         MSLL[0], BIC, skew, kurtosis]

display(blr_metrics)

blr_metrics.to_csv(os.path.join(out_dir,'blr_metrics.csv'))

## Questions to discuss
1. Model selection: Which model selection criteria would you use to choose the optimal model?
2. Model flexibility: What happens when you change the warping or B-spline settings?
3. Bias-variance tradeoff: How would you consider the bias-variance tradeoff when deciding the models parameters?
4. Which independent variables do you think are important to add to the normative model?
5. Are there other model selection criteria that you think should be considered?

## Suggested further readings

1. [PCNtoolkit Background](https://pcntoolkit.readthedocs.io/en/latest/pages/pcntoolkit_background.html)
2. [Conceptualizing mental disorders as deviations from normative functioning](https://www.nature.com/articles/s41380-019-0441-1)
3. [Understanding Heterogeneity in Clinical Cohorts Using Normative Models: Beyond Case-Control Studies
](https://www.sciencedirect.com/science/article/pii/S0006322316000020)


In [32]:
# which sex do we want to plot?
sex = 1 # 1 = male 0 = female
if sex == 1:
    clr = 'blue';
else:
    clr = 'red'

# create dummy data for visualisation
print('configuring dummy data ...')
xx = np.arange(xmin, xmax, 0.5)
X0_dummy = np.zeros((len(xx), 2))
X0_dummy[:,0] = xx
X0_dummy[:,1] = sex

# create the design matrix
X_dummy = create_design_matrix(X0_dummy, xmin=xmin, xmax=xmax, site_ids=None ,p=3 ,nknots=4 ,all_sites=site_ids)

# save the dummy covariates
cov_file_dummy = os.path.join(out_dir,'cov_bspline_dummy_mean.txt')
np.savetxt(cov_file_dummy, X_dummy)

configuring dummy data ...


In [ ]:
sns.set(style='white')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']

for idp_num, idp in enumerate(idp_ids):
    #idp_short = extract_relevant_part(idp)
    print('Running IDP', idp_num, idp, ':')

    idp_dir = os.path.join(out_dir, idp)
    os.chdir(idp_dir)

    # load the true data points
    yhat_te = load_2d(os.path.join(idp_dir, 'yhat_estimate.txt'))
    s2_te = load_2d(os.path.join(idp_dir, 'ys2_estimate.txt'))
    y_te = load_2d(os.path.join(idp_dir, 'resp_te.txt'))

    plt.figure(figsize=(3, 3))

    # set up the covariates for the dummy data
    print('Making predictions with dummy covariates (for visualisation)')
    yhat, s2 = predict(cov_file_dummy,
                       alg = 'blr',
                       respfile = None,
                       model_path = os.path.join(idp_dir,'Models'),
                       outputsuffix = '_dummy')

    # load the normative model
    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle)

    # get the warp and warp parameters
    W = nm.blr.warp
    warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1]

    # first, we warp predictions for the true data and compute evaluation metrics
    med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
    med_te = med_te[:, np.newaxis]
    print('metrics:', evaluate(y_te, med_te))

    # then, we warp dummy predictions to create the plots
    med, pr_int = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param)

    # extract the different variance components to visualise
    beta, junk1, junk2 = nm.blr._parse_hyps(nm.blr.hyp, X_dummy)
    s2n = 1/beta # variation (aleatoric uncertainty)
    s2s = s2-s2n # modelling uncertainty (epistemic uncertainty)

    # plot the data points
    y_te_rescaled_all = np.zeros_like(y_te)
    for sid, site in enumerate(site_ids):
        # plot the true test data points
        if all(elem in site_ids for elem in site_ids):
            # all data in the test set are present in the training set

            # first, we select the data points belonging to this particular site
            idx = np.where(np.bitwise_and(X_te[:,2] == sex, X_te[:, sid + len(cols_cov) + 1] != 0))[0]
            if len(idx) == 0:
                print('No data for site', sid, site, 'skipping...')
                continue
            idx = idx[idx < len(y_te)]
            # then directly adjust the data
            idx_dummy = np.bitwise_and(X_dummy[:,1] > X_te[idx,1].min(), X_dummy[:,1] < X_te[idx,1].max())
            y_te_rescaled = y_te[idx] - np.median(y_te[idx]) + np.median(med[idx_dummy])
        else:
            # we need to adjust the data based on the adaptation dataset

            # first, select the data point belonging to this particular site
            idx = np.where(np.bitwise_and(X_te[:,2] == sex, (df_te['site'] == site).to_numpy()))[0]

            # load the adaptation data
            y_ad = load_2d(os.path.join(idp_dir, 'resp_ad.txt'))
            X_ad = load_2d(os.path.join(idp_dir, 'cov_bspline_ad.txt'))
            idx_a = np.where(np.bitwise_and(X_ad[:,2] == sex, (df_tr['site'] == site).to_numpy()))[0]
            if len(idx) < 2 or len(idx_a) < 2:
                print('Insufficent data for site', sid, site, 'skipping...')
                continue

            # adjust and rescale the data
            y_te_rescaled, s2_rescaled = nm.blr.predict_and_adjust(nm.blr.hyp,
                                                                   X_ad[idx_a,:],
                                                                   np.squeeze(y_ad[idx_a]),
                                                                   Xs=None,
                                                                   ys=np.squeeze(y_te[idx]))
        # plot the (adjusted) data points
        plt.scatter(X_te[idx,1], y_te_rescaled, s=5, color=clr, alpha = 0.1)

    # plot the median of the dummy data
    plt.plot(xx, med, clr)

    # fill the gaps in between the centiles
    junk, pr_int25 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.05,0.95])
    #junk, pr_int99 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25[:,0], pr_int25[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int95[:,0], pr_int95[:,1], alpha = 0.1,color=clr)
    #plt.fill_between(xx, pr_int99[:,0], pr_int99[:,1], alpha = 0.1,color=clr)

    # make the width of each centile proportional to the epistemic uncertainty
    junk, pr_int25l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.05,0.95])
    #junk, pr_int99l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.01,0.99])
    junk, pr_int25u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.05,0.95])
    #junk, pr_int99u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25l[:,0], pr_int25u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,0], pr_int95u[:,0], alpha = 0.3,color=clr)
    #plt.fill_between(xx, pr_int99l[:,0], pr_int99u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int25l[:,1], pr_int25u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,1], pr_int95u[:,1], alpha = 0.3,color=clr)
    #plt.fill_between(xx, pr_int99l[:,1], pr_int99u[:,1], alpha = 0.3,color=clr)

    # plot actual centile lines
    plt.plot(xx, pr_int25[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int25[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,1],color=clr, linewidth=0.5)
    #plt.plot(xx, pr_int99[:,0],color=clr, linewidth=0.5)
    #plt.plot(xx, pr_int99[:,1],color=clr, linewidth=0.5)

    plt.xlabel('Age', fontweight='bold')
    plt.ylabel('GMV', fontweight='bold')
    plt.title(idp, fontweight='bold')
    plt.xlim((45,83))
    plt.savefig(os.path.join(idp_dir, 'centiles_' + str(sex) + '.svg'), bbox_inches='tight', transparent=True, format='svg', dpi=300)
    plt.close()
    #plt.show()

os.chdir(out_dir)